In [1]:
if 'google.colab' in str(get_ipython()):
    print('Cloning code base to colab....')
    !git clone https://github.com/Spandan-Madan/generalization_to_OOD_category_viewpoint_cominations.git
    !cd generalization_to_OOD_category_viewpoint_cominations/utils && bash download_mnist_rotation.sh
    CODE_ROOT = "generalization_to_OOD_category_viewpoint_cominations/"
else:
    CODE_ROOT = '..'

Cloning code base to colab....
fatal: destination path 'generalization_to_OOD_category_viewpoint_cominations' already exists and is not an empty directory.
--2022-04-04 01:43:54--  https://www.dropbox.com/s/wdws3b3fjo190sk/self_generated.tar.gz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wdws3b3fjo190sk/self_generated.tar.gz [following]
--2022-04-04 01:43:54--  https://www.dropbox.com/s/raw/wdws3b3fjo190sk/self_generated.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc43bb78091a5973d181af398193.dl.dropboxusercontent.com/cd/0/inline/Bis0oOO1swWkp4GQQUPsdLk28JrH4YJ6eT3vzVYYcgtTpYn5HgfoYub45MYEl5NDsynUG81oze6AzI2mlGGE6hkzPQnyAnDbuy_ua4J-jxHN1rPIHSlGRR3qBSCnhr22jgm97UazUKtoUSDG1xwqoLlcv-HTpWKcvwmFDrH9Af

In [2]:
!pip install datasets
!pip install transformers

In [3]:
!ls

!pwd

generalization_to_OOD_category_viewpoint_cominations  sample_data
/content


In [4]:
cd /content/generalization_to_OOD_category_viewpoint_cominations/res

/content/generalization_to_OOD_category_viewpoint_cominations/res


In [5]:
!pwd
!ls

/content/generalization_to_OOD_category_viewpoint_cominations/res
loader	loss  metrics  models


In [7]:
#https://huggingface.co/blog/fine-tune-vit

import datasets 
import transformers

CODE_ROOT = '..'
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
import random
ImageFile.LOAD_TRUNCATED_IMAGES = True
import argparse
import pickle
import sys
sys.path.append('%s/res/'%CODE_ROOT)
from models.models import get_model
from loader.loader import get_loader
from tqdm.notebook import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
#load MNIST dataset and setup

DATASET_NAME = 'mnist_rotation_six_by_nine'

NUM_EPOCHS = 10
BATCH_SIZE = 1

ARCH = 'LATE_BRANCHING_COMBINED'

image_transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])


GPU = 1

In [9]:
NUM_CLASSES = (10,10,10,10)
loader_new = get_loader('multi_attribute_loader_file_list_mnist_rotation')

file_list_root = '%s/dataset_lists/mnist_rotation_lists/'%CODE_ROOT
att_path = '%s/dataset_lists/combined_attributes.p'%CODE_ROOT
shuffles = {'train':True,'val':True,'test':False}
data_dir = '%s/data/'%CODE_ROOT

In [10]:
file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}

for phase in ['train','val','test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,DATASET_NAME)
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform, data_dir)
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=BATCH_SIZE, shuffle = shuffles[phase], num_workers=2,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])

In [11]:
#https://danielvanstrien.xyz/metadata/deployment/huggingface/ethics/huggingface-datasets/faiss/2022/01/13/image_search.html
#create the dataframes for the ViT

from torchvision.transforms import ToPILImage
from PIL import Image
import pandas as pd
import io
from collections import defaultdict

train_data = defaultdict(list)

for data in tqdm(dset_loaders['train']):
    inputs, labels_all, paths = data
    
    train_data['image_file_path'].append(str(paths[0]))
    train_data['image'].append(os.path.basename(str(paths[0])))
    train_data['labels'].append(labels_all.data.numpy()[0][1])
    
val_data = defaultdict(list)

for data in tqdm(dset_loaders['val']):
    inputs, labels_all, paths = data
    
    val_data['image_file_path'].append(str(paths[0]))
    val_data['image'].append(os.path.basename(str(paths[0])))
    val_data['labels'].append(labels_all.data.numpy()[0][1])
    
test_data = defaultdict(list)

for data in tqdm(dset_loaders['test']):
    inputs, labels_all, paths = data
    
    test_data['image_file_path'].append(str(paths[0]))
    test_data['image'].append(os.path.basename(str(paths[0])))
    test_data['labels'].append(str(labels_all.data.numpy()[0][1]))

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [12]:
#create dataset objects, and make class labels

from datasets import Dataset
from datasets import ClassLabel
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

train_dataset.features['labels'] = ClassLabel(num_classes=10, names=['0','1','2','3','4','5','6','7','8','9'], id=None)
train_dataset.features['labels']

ClassLabel(num_classes=10, names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], id=None)

In [18]:
!pip install python-resize-image
!pip install BytesIO

ERROR: Could not find a version that satisfies the requirement BytesIO (from versions: none)
ERROR: No matching distribution found for BytesIO


In [20]:
#function to load and reformat image

import resizeimage
from io import BytesIO
def load_image(path):
    
    img = Image.open(path)
    img = img.convert("RGB")
    b = BytesIO()
    img.save(b,format="jpeg")
    im = Image.open(b)
        
    return im 

In [22]:
#function to load image

def try_load_image(filename):
    try:
        image = load_image(filename)
        if isinstance(image, Image.Image):
            return image
    except Image.UnidentifiedImageError:
        return None

In [23]:
#load the images from supplied paths

train_dataset = train_dataset.map(lambda example: {"img": try_load_image(example['image_file_path'])},writer_batch_size=50)
val_dataset = val_dataset.map(lambda example: {"img": try_load_image(example['image_file_path'])},writer_batch_size=50)
test_dataset = test_dataset.map(lambda example: {"img": try_load_image(example['image_file_path'])},writer_batch_size=50)


  0%|          | 0/50000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

In [24]:
train_dataset

Dataset({
    features: ['image_file_path', 'image', 'labels', 'img'],
    num_rows: 50000
})

In [25]:
ex = train_dataset[400]
ex

{'image': '8_7_59909.png',
 'image_file_path': '../data//mnist_rotation/self_generated/five_by_nine_seen_images/8_7_59909.png',
 'img': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=28x28 at 0x7FD2F069DA10>,
 'labels': 7}

In [26]:
image = ex['img']
print(image)
image

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=28x28 at 0x7FD2F069DA10>


In [27]:
labels = train_dataset.features['labels']
labels

ClassLabel(num_classes=10, names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], id=None)

In [28]:
labels.int2str(ex['labels'])


'7'

In [29]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
print(feature_extractor)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}



In [30]:
print(image)
feature_extractor(image, return_tensors='pt')


<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=28x28 at 0x7FD2F069DA10>


{'pixel_values': tensor([[[[-0.8510, -0.8510, -0.8510,  ..., -1.0000, -1.0000, -1.0000],
          [-0.8510, -0.8510, -0.8510,  ..., -1.0000, -1.0000, -1.0000],
          [-0.8510, -0.8510, -0.8510,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-0.8510, -0.8510, -0.8510,  ..., -1.0000, -1.0000, -1.0000],
          [-0.8510, -0.8510, -0.8510,  ..., -1.0000, -1.0000, -1.0000],
          [-0.8510, -0.8510, -0.8510,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-0.8510, -0.8510, -0.8510,  ..., -1.0000, -1.0000, -1.0000],
          [-0

In [31]:
def process_example(example):
    inputs = feature_extractor(example['img'], return_tensors='pt')
    inputs['labels'] = example['labels']
    return inputs

In [32]:
process_example(train_dataset[0])


{'pixel_values': tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]]), 'labels': 4}

In [33]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['img']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['labels']
    return inputs

In [34]:
prepared_ds_train = train_dataset.with_transform(transform)
prepared_ds_val = val_dataset.with_transform(transform)
prepared_ds_test = test_dataset.with_transform(transform)


In [35]:
prepared_ds_train[0:2]


{'pixel_values': tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1

In [36]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [37]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [38]:
from transformers import ViTForImageClassification

#labels = prepared_ds_train.features['labels'].names
labels = train_dataset.features['labels'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds_train,
    eval_dataset=prepared_ds_val,
    tokenizer=feature_extractor,
)

Using amp half precision backend


In [41]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 50000
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12500


Step,Training Loss,Validation Loss,Accuracy
100,1.246600,1.099151,0.508700
200,0.880900,0.873893,0.597400
300,0.908900,0.775531,0.655900
400,0.773000,0.644515,0.753600
500,0.321900,0.378654,0.858500
600,0.577700,0.376270,0.866200
700,0.305600,0.316457,0.899900
800,0.396300,0.313814,0.899500
900,0.188200,0.180731,0.947600
1000,0.365100,0.393753,0.878300


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to ./vit-base-beans/checkpoint-100
Configuration saved in ./vit-base-beans/checkpoint-100/config.json
Model weights saved in ./vit-base-beans/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./vit-base-beans/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to ./vit-base-beans/checkpoint-200
Configuration saved in ./vit-base-beans/checkpoint-200/config.json
Model weights saved in ./vit-base-beans/checkpoint-200/pytorch_model.bin
Feature extractor saved in ./vit-base-beans/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to ./vit-base-beans/checkpoint-300
Configuration saved in ./vit-base-beans/checkpoint-300/config.json
Model weights saved in ./vit-base-beans/checkpoint-300/pytorch_model.bin
Feature extractor save

***** train metrics *****
  epoch                    =           4.0
  total_flos               = 14435042795GF
  train_loss               =        0.0993
  train_runtime            =    2:20:36.66
  train_samples_per_second =        23.706
  train_steps_per_second   =         1.482


In [42]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

NameError: ignored